# ✅ Step 0. 설치 & 공용 임포트

In [1]:
# ✅ 필수 라이브러리 설치(노트북 첫 셀)
# - transformers / datasets : 모델 & 데이터셋
# - ragas : RAG 품질평가
# - langchain-openai : RAGAS 채점 LLM로 OpenAI 사용
# - torch : PyTorch (Colab GPU 사용 시 자동 설치되어 있는 경우도 많음)
%pip -q install "transformers>=4.41" "datasets>=2.19" "ragas>=0.1.21" "langchain-openai>=0.1.17" "torch>=2.1,<3"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.2/277.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.9/148.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
# ✅ 공용 임포트(두 번째 셀)
from typing import List, Dict, Any, Tuple
import os
import torch
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
from langchain_openai import ChatOpenAI

# ✅ Step 1. OpenAI API 키 (Colab 전용 방식 고정)

In [4]:
# ✅ google.colab.userdata 사용
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

# (선택) 환경변수로도 등록해두면, 라이브러리가 자동 인식해
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY if OPENAI_API_KEY else os.environ.get("OPENAI_API_KEY", "")

# 키 존재 간단 확인(유출 방지: 길이만 출력)
print("OPENAI_API_KEY loaded:", "YES" if os.environ.get("OPENAI_API_KEY") else "NO")


OPENAI_API_KEY loaded: YES


# ✅ Step 2. 데이터셋 로딩 (HF Hub)


In [20]:
# ✅ HF 데이터셋 로딩
def load_hf_split(name: str = "sssssungjae/finance-kb-mixed-dataset-final", split: str = "eval"):
    """
    HF Hub에서 지정 split 로딩.
    - 반환: datasets.Dataset
    """
    return load_dataset(name, split=split)

raw_ds = load_hf_split()
raw_ds


Dataset({
    features: ['text', '__index_level_0__'],
    num_rows: 1055
})

# ✅ Step 3. Qwen 포맷 파싱 → question / ground_truth / contexts

Qwen 채팅 데이터는 보통 <|im_start|>user ... <|im_end|> 같은 템플릿을 써. 안전하게 파싱하고, RAGAS 스키마(questions, answers, contexts, ground_truth)에 맞춰 컬럼을 정리해. (chat 템플릿 개념은 HF 공식 가이드 참고)
Hugging Face

In [21]:
def parse_qwen_chat_field(hf_ds) -> Dataset:
    """
    raw_ds['text'] 안의 Qwen 채팅 문자열에서
    - question : user 블록
    - ground_truth : assistant 블록
    를 추출하고, RAGAS 입력용 contexts(List[List[str]]) 더미 컬럼을 추가한다.
    """
    def _parse(row):
        t = row.get("text", "")
        q, a = None, None
        try:
            # 매우 보편적인 Qwen 포맷 기준 파싱
            q = t.split("<|im_start|>user")[1].split("<|im_end|>")[0].strip()
            a = t.split("<|im_start|>assistant")[1].split("<|im_end|>")[0].strip()
        except Exception:
            pass
        return {"question": q, "ground_truth": a}

    ds = hf_ds.map(_parse, remove_columns=[c for c in hf_ds.column_names if c != "text"])
    ds = ds.filter(lambda r: bool(r["question"]) and bool(r["ground_truth"]))
    ds = ds.add_column("contexts", [[""]] * len(ds))  # RAG이 아니므로 더미 context
    # 불필요한 원문 text 제거(선택)
    if "text" in ds.column_names:
        ds = ds.remove_columns(["text"])
    return ds

parsed_ds = parse_qwen_chat_field(raw_ds)
parsed_ds.select(range(min(3, len(parsed_ds))))


Dataset({
    features: ['question', 'ground_truth', 'contexts'],
    num_rows: 3
})

In [22]:
# --- Step 3.5: 평가 샘플 500개로 제한(재현성 위해 시드 고정)
SUBSET_SIZE = 500  # ← 네가 원하는 개수
parsed_ds = parsed_ds.shuffle(seed=42).select(range(min(SUBSET_SIZE, len(parsed_ds))))
print(f"[subset] {len(parsed_ds)} rows will be evaluated")


[subset] 500 rows will be evaluated


# ✅ Step 4. 모델/토크나이저 로딩 (Qwen Instruct)

Qwen Instruct 모델은 chat 템플릿으로 프롬프트를 만들고, 배치 생성 시 왼쪽 패딩이 안전해. (apply_chat_template / padding_side 권장사항)


In [23]:
def load_qwen(model_id: str = "Qwen/Qwen2.5-0.5B-Instruct"):
    """
    Qwen Instruct 모델과 토크나이저 로딩.
    - dtype은 bfloat16이 안될 때 float16으로 폴백.
    - decoder-only이므로 padding_side='left' 권장.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    try:
        model = AutoModelForCausalLM.from_pretrained(
            model_id, device_map="auto", torch_dtype=torch.bfloat16
        )
    except Exception:
        model = AutoModelForCausalLM.from_pretrained(
            model_id, device_map="auto", dtype=torch.float16
        )
    tokenizer.padding_side = "left"  # 배치 길이가 다를 때 생성 안정성 ↑
    return model, tokenizer

model, tokenizer = load_qwen()
model.generation_config.max_new_tokens = 128  # 기본값 (필요 시 수정)
"Model & tokenizer ready"


'Model & tokenizer ready'

# ✅ Step 5. Qwen 채팅 프롬프트 구성 (apply_chat_template)

메시지 리스트(roles: system/user/assistant)를 apply_chat_template(..., tokenize=False, add_generation_prompt=True)로 문자열 변환 후 토크나이즈해. (HF 공식 문서 권장 패턴)

In [24]:
def build_qwen_prompts(tokenizer, questions: List[str]) -> List[str]:
    """
    Qwen chat 템플릿 문자열 생성.
    - system: 모델 역할 규정
    - user: 실제 질문
    - add_generation_prompt=True : assistant 응답 자리까지 포함
    """
    prompts = []
    for q in questions:
        messages = [
            {"role": "system", "content": "You are a helpful banking assistant."},
            {"role": "user", "content": q},
        ]
        s = tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        prompts.append(s)
    return prompts

chat_prompts = build_qwen_prompts(tokenizer, parsed_ds["question"])
chat_prompts[0][:200] + " ..."


'<|im_start|>system\nYou are a helpful banking assistant.<|im_end|>\n<|im_start|>user\n임원 또는 주요주주가 특정증권의 소유 변동사항을 보고하지 않았을 경우, 어떠한 법적 처벌을 받을 수 있는가?<|im_end|>\n<|im_start|>assistant\n ...'

# ✅ Step 6. 배치 생성 (재현성 위해 sampling off)


In [25]:
# ---- Step 6. 배치 생성(재현성 + tqdm + 속도 로깅) ----
from typing import List
from transformers import GenerationConfig
from tqdm import tqdm
import time
import torch

def batch_generate(
    model, tokenizer, chat_prompts: List[str],
    batch_size: int = 8, max_new_tokens: int = 128
) -> List[str]:
    """
    프롬프트 문자열 리스트를 배치로 생성.
    - do_sample=False : 평가 재현성↑
    - tqdm로 진행률/ETA 표시
    - 토큰/초 속도 로깅
    - add_special_tokens=False (chat 템플릿 문자열 재토크나이즈 시 중복특수토큰 방지)
    - torch.inference_mode()로 약간의 이득
    """
    # --- 안전장치 ---
    if hasattr(tokenizer, "padding_side") and tokenizer.padding_side != "left":
        tokenizer.padding_side = "left"   # 디코더 전용 권장값  :contentReference[oaicite:7]{index=7}

    # KV cache가 꺼져 있지 않은지(꺼져 있으면 매우 느려짐)  :contentReference[oaicite:8]{index=8}
    use_cache = getattr(model.generation_config, "use_cache", True)
    if not use_cache:
        try:
            model.generation_config.use_cache = True
        except Exception:
            pass

    # --- 생성 설정(샘플링 플래그 제거) ---
    gen_cfg = GenerationConfig(
        do_sample=False,
        num_beams=1,
        max_new_tokens=max_new_tokens,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        use_cache=True,
    )
    for k in ("temperature", "top_p", "top_k", "typical_p", "penalty_alpha", "min_p"):
        if hasattr(gen_cfg, k):
            try: setattr(gen_cfg, k, None)
            except: pass

    outs: List[str] = []
    total_gen_tokens = 0
    total_time = 0.0

    # --- 진행바 ---
    pbar = tqdm(range(0, len(chat_prompts), batch_size), desc=f"gen bs={batch_size} max_new={max_new_tokens}")

    with torch.inference_mode():
        for i in pbar:
            batch = chat_prompts[i:i+batch_size]

            # 토크나이즈: 템플릿 문자열 → 토큰 (중복특수토큰 방지)  :contentReference[oaicite:9]{index=9}
            inputs = tokenizer(
                batch, return_tensors="pt", padding=True, truncation=True, add_special_tokens=False
            ).to(model.device)

            # 생성
            t0 = time.monotonic()
            outputs = model.generate(**inputs, generation_config=gen_cfg)
            dt = time.monotonic() - t0

            # 생성 토큰만 추출
            gen_only = outputs[:, inputs.input_ids.shape[1]:]
            answers = tokenizer.batch_decode(gen_only, skip_special_tokens=True)
            outs.extend(answers)

            # 속도 로깅(토큰/초)
            gen_tokens = int(gen_only.shape[0] * gen_only.shape[1])  # (batch * 생성길이)
            total_gen_tokens += gen_tokens
            total_time += dt
            tps = (gen_tokens / dt) if dt > 0 else 0.0
            pbar.set_postfix(tps=f"{tps:.1f}", avg_tps=f"{(total_gen_tokens/total_time):.1f}")

    print(f"[done] total_gen_tokens={total_gen_tokens}, avg_tokens_per_sec={total_gen_tokens/total_time:.1f}")
    return outs

generated_answers = batch_generate(model, tokenizer, chat_prompts, batch_size=8, max_new_tokens=128)
len(generated_answers), generated_answers[0][:200] + " ..."


gen bs=8 max_new=128: 100%|██████████| 63/63 [07:10<00:00,  6.83s/it, avg_tps=148.9, tps=70.1]

[done] total_gen_tokens=64000, avg_tokens_per_sec=148.9


(500,
 '임원 또는 주요주주가 특정 증권의 소유 변동사항을 보고하지 않았을 경우, 그들이 이에 대한 책임을 지지 않거나, 그들의 책임을 인정하지 않으면 다음과 같은 법적 처벌을 받을 수 있습니다:\n\n1. **죄송한 책임**: 이는 주주가 증권에 대한 이해를 부족하거나, 증권에 대한 정보를 제공하지 못한 경우에 발생할 수 있습니다. 주주가 증권에 대한 이해를 부족한  ...')

# ✅ Step 7. RAGAS 입력 데이터셋 구성

In [26]:
def to_ragas_dataset(
    questions: List[str], answers: List[str], contexts: List[List[str]], gts: List[str]
) -> Dataset:
    """
    RAGAS evaluate()가 요구하는 컬럼 스키마로 변환.
    - question / answer / contexts(List[List[str]]) / ground_truth
    """
    return Dataset.from_dict({
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truth": gts,
    })

ragas_ds = to_ragas_dataset(
    questions=parsed_ds["question"],
    answers=generated_answers,
    contexts=parsed_ds["contexts"],
    gts=parsed_ds["ground_truth"],
)
ragas_ds


Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 500
})

In [29]:
# --- Step 7.9: 평가용으로 500개만 강제 ---
SUBSET_SIZE = 500
ragas_ds_500 = ragas_ds.shuffle(seed=42).select(range(min(SUBSET_SIZE, len(ragas_ds))))  # HF Datasets 공식 패턴
print("len(ragas_ds_500) =", len(ragas_ds_500))  # → 500이 떠야 정상


len(ragas_ds_500) = 500


# ✅ Step 8. RAGAS 평가 실행 (온도 0 고정)

ragas.evaluate()에 데이터셋과 지표 목록, 채점용 LLM을 넘겨. 평가 지표는 faithfulness / answer_relevancy / context_precision / context_recall 예시를 사용했고, 채점 LLM은 ChatOpenAI를 temperature=0으로 고정. (RAGAS evaluate & 지표 문서, LangChain ChatOpenAI)

In [31]:
# ---- Step 8. 비용·시간 최적화 평가 (복붙) ----
from langchain_openai import ChatOpenAI
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
from ragas.run_config import RunConfig
from ragas.cost import get_token_usage_for_openai

# 0) 평가 지표 최소화: 생성 품질 핵심 2종만 사용(비용/시간 ↓)
#    * 필요 시 CONTEXT_METRICS=True로 바꿔 컨텍스트 정밀/재현도 추가
CONTEXT_METRICS = False
METRICS = [faithfulness, answer_relevancy] if not CONTEXT_METRICS \
          else [faithfulness, answer_relevancy, context_precision, context_recall]

# 1) 데이터셋 500개로 강제 (앞에서 ragas_ds_500 만들었어도 안전하게 한 번 더 고정)
SUBSET_SIZE = 500
ragas_ds_eval = ragas_ds.select(range(min(SUBSET_SIZE, len(ragas_ds)))) \
                if "ragas_ds_500" not in globals() else ragas_ds_500
print(f"[eval] rows={len(ragas_ds_eval)} | metrics={len(METRICS)} → jobs={len(ragas_ds_eval)*len(METRICS)}")

# 2) 저가 채점 LLM(gpt-4o-mini) + 동시성 설정
#    - 공식 단가(텍스트): 입력 $0.60/M, 출력 $2.40/M 토큰 (2025-09 기준). :contentReference[oaicite:0]{index=0}
judge = ChatOpenAI(model="gpt-4o-mini", temperature=0)
rc = RunConfig(max_workers=4)   # 동시성 4로 완만하게(레이트리밋/스파이크 방지). 값 키우면 벽시계 시간↓. :contentReference[oaicite:1]{index=1}

# 3) 평가 실행 (진행바 헷갈리면 show_progress=False)
result = evaluate(
    dataset=ragas_ds_eval,                  # ← 반드시 500 샘플 세트로
    metrics=METRICS,
    llm=judge,
    run_config=rc,                          # 동시성/타임아웃/재시도 등의 런 컨트롤. :contentReference[oaicite:2]{index=2}
    token_usage_parser=get_token_usage_for_openai,  # 토큰 집계/비용 추정용. :contentReference[oaicite:3]{index=3}
    batch_size=8,
    show_progress=True
)

# 4) 점수 + 토큰/비용 추정 출력
summary = {k: round(float(v), 3) for k, v in result.items()}
print("scores:", summary)

usage = result.total_tokens()   # TokenUsage(input_tokens=..., output_tokens=..., model='...')
# 공식 단가 반영(입력 $0.15/M, 출력 $0.6/M)  :contentReference[oaicite:4]{index=4}
est_cost = result.total_cost(
    cost_per_input_token=0.15/1e6,
    cost_per_output_token=0.6/1e6
)
print(f"[tokens] in={usage.input_tokens}, out={usage.output_tokens}")
print(f"[est cost] ≈ ${est_cost:.4f} (model=gpt-4o-mini)")


[eval] rows=500 | metrics=2 → jobs=1000


Evaluating:   0%|          | 0/1000 [00:00<?, ?it/s]

Batch 1/125:   0%|          | 0/8 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[125]: IndexError(list index out of range)
ERROR:ragas.executor:Exception raised in Job[397]: IndexError(list index out of range)
ERROR:ragas.executor:Exception raised in Job[669]: IndexError(list index out of range)
ERROR:ragas.executor:Exception raised in Job[853]: IndexError(list index out of range)


AttributeError: 'EvaluationResult' object has no attribute 'items'

In [33]:
# --- Step 8 결과 요약/비용 계산 (EvaluationResult -> DataFrame 경로) ---

# 1) 상세 테이블로 변환
df = result.to_pandas()  # 각 행별 점수 + 원본 컬럼 합쳐진 DF  :contentReference[oaicite:2]{index=2}
print("[detail] shape:", df.shape)
display(df.head(3))

# 2) 지표 컬럼 자동 추출(숫자형만)
import pandas as pd
metric_cols = [c for c in df.columns if df[c].dtype.kind in "fi" and c not in ("row_index",)]
print("[metrics]", metric_cols)

# 3) NaN 개수 확인(에러 난 row는 여기 NaN으로 옴)
print("[nan count per metric]")
print(df[metric_cols].isna().sum())

# 4) 요약 점수(평균) 계산
summary = df[metric_cols].mean(numeric_only=True).to_dict()
summary = {k: round(float(v), 3) for k, v in summary.items()}
print("[summary]", summary)

# 5) CSV 저장
df.to_csv("ragas_detail.csv", index=False)
print("Saved: ragas_detail.csv")

# 6) 토큰/비용(옵션: token_usage_parser 넣었을 때만 가능)
try:
    usage = result.total_tokens()   # TokenUsage(in_tokens, out_tokens, model=...)  :contentReference[oaicite:3]{index=3}
    est_cost = result.total_cost(   # 단가 직접 넣어 계산  :contentReference[oaicite:4]{index=4}
        cost_per_input_token=0.15/1e6,
        cost_per_output_token=0.60/1e6
    )
    print(f"[tokens] in={usage.input_tokens}, out={usage.output_tokens}")
    print(f"[est cost] ≈ ${est_cost:.4f} (model=gpt-4o-mini)")
except Exception as e:
    print("cost/usage not available:", e)


[detail] shape: (500, 6)


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy
0,미국의 정상무역관계(NTR) 용어가 의미하는 바는 무엇인가요?,[],"미국의 정상무역관계(National Trade Relations, NTR)는 미국과...",<think>\n\n</think>\n\n정상무역관계(Normal Trade Rel...,0.4,0.866647
1,"호가가격은 어떻게 결정되며, 이사회 결의 주식수에 따른 제한은 무엇인가요?",[],호가가격은 주식의 가치를 결정하는 주요 요인 중 하나입니다. 주가는 주식의 가치를 ...,<think>\n\n</think>\n\n호가가격과 이사회 결의에 따른 주식수 제한...,0.0,0.000000
2,정말 눈으로 보지 않고 믿음으로 걷는다는 것은 무엇을 의미할까요?,[],"""정말 눈으로 보지 않고 믿음으로 걷는다는 것은 믿음의 강도를 느끼는 것""을 의미합...",<think>\n\n</think>\n\no 눈으로 보지 말고 믿음으로 걷는다는 것...,0.0,0.830806


[metrics] ['faithfulness', 'answer_relevancy']
[nan count per metric]
faithfulness        0
answer_relevancy    4
dtype: int64
[summary] {'faithfulness': 0.011, 'answer_relevancy': 0.603}
Saved: ragas_detail.csv
[tokens] in=907942, out=314676
[est cost] ≈ $0.3250 (model=gpt-4o-mini)


# ✅ Step 9. 결과 보기 & 저장(선택)

In [34]:
# --- Step 9. EvaluationResult -> DataFrame 요약/저장/비용 계산 ---

# 1) 상세 결과 DataFrame으로 변환
df = result.to_pandas()  # EvaluationResult 표준 경로
print("[detail] shape:", df.shape)
try:
    display(df.head(3))
except Exception:
    print(df.head(3))

# 2) 메트릭 컬럼 자동 탐지(숫자형)
metric_cols = [c for c in df.columns if df[c].dtype.kind in "fi"]
print("[metrics]", metric_cols)

# 3) NaN (채점 실패 샘플) 카운트
print("[nan count]")
print(df[metric_cols].isna().sum())

# 4) 요약 점수(평균)
summary = df[metric_cols].mean(numeric_only=True).to_dict()
summary = {k: round(float(v), 3) for k, v in summary.items()}
print("[summary]", summary)

# 5) CSV 저장
df.to_csv("ragas_detail.csv", index=False)
print("Saved: ragas_detail.csv")

# 6) 토큰/비용 (RAGAS 실행 시 token_usage_parser를 넣었다면)
try:
    usage = result.total_tokens()   # TokenUsage(in/out/model)
    # ✅ gpt-4o-mini 공식 단가(입력 $0.15/M, 출력 $0.60/M) 반영
    est_cost = result.total_cost(
        cost_per_input_token=0.15/1e6,
        cost_per_output_token=0.60/1e6
    )
    print(f"[tokens] in={usage.input_tokens}, out={usage.output_tokens}")
    print(f"[est cost] ≈ ${est_cost:.4f} (model=gpt-4o-mini)")
except Exception as e:
    print("cost/usage not available:", e)


[detail] shape: (500, 6)


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy
0,미국의 정상무역관계(NTR) 용어가 의미하는 바는 무엇인가요?,[],"미국의 정상무역관계(National Trade Relations, NTR)는 미국과...",<think>\n\n</think>\n\n정상무역관계(Normal Trade Rel...,0.4,0.866647
1,"호가가격은 어떻게 결정되며, 이사회 결의 주식수에 따른 제한은 무엇인가요?",[],호가가격은 주식의 가치를 결정하는 주요 요인 중 하나입니다. 주가는 주식의 가치를 ...,<think>\n\n</think>\n\n호가가격과 이사회 결의에 따른 주식수 제한...,0.0,0.000000
2,정말 눈으로 보지 않고 믿음으로 걷는다는 것은 무엇을 의미할까요?,[],"""정말 눈으로 보지 않고 믿음으로 걷는다는 것은 믿음의 강도를 느끼는 것""을 의미합...",<think>\n\n</think>\n\no 눈으로 보지 말고 믿음으로 걷는다는 것...,0.0,0.830806


[metrics] ['faithfulness', 'answer_relevancy']
[nan count]
faithfulness        0
answer_relevancy    4
dtype: int64
[summary] {'faithfulness': 0.011, 'answer_relevancy': 0.603}
Saved: ragas_detail.csv
[tokens] in=907942, out=314676
[est cost] ≈ $0.3250 (model=gpt-4o-mini)
